In [1]:
import pandas as pd
import numpy as np
import pyod
np.set_printoptions(precision=3)

In [28]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.knn import KNN
from pyod.models.xgbod import XGBOD

In [41]:
training_set = pd.read_csv('Balanced_Samples/Sample_100K.csv',sep=',',header=0)
testing_set = pd.read_csv('Unbalanced_Samples/Sample_60K.csv',sep=",",header=0)

C:\Users\caioa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
def const_columns():
    return 77

trainingSet = training_set.values
testingSet = testing_set.values

#print(testingSet[:,84])
for i in range (100000):
    if(trainingSet[i, 84] == "ddos"):
        trainingSet[i, 84] = 1
    else:
        trainingSet[i, 84] = 0
        
for i in range (60000):
    if(testingSet[i, 84] == "ddos"):
        testingSet[i, 84] = 1
    else:
        testingSet[i, 84] = 0

#np.random.shuffle(testingSet)


testingSet = np.delete(testingSet, 16, axis=1)
testingSet = np.delete(testingSet, 17, axis=1)
testingSet = np.delete(testingSet, 7, axis=1)
testingSet = np.delete(testingSet, 4, axis=1)
testingSet = np.delete(testingSet, 2, axis=1)
testingSet = np.delete(testingSet, 1, axis=1)
testingSet = np.delete(testingSet, 0, axis=1)

trainingSet = np.delete(trainingSet, 16, axis=1)
trainingSet = np.delete(trainingSet, 17, axis=1)
trainingSet = np.delete(trainingSet, 7, axis=1)
trainingSet = np.delete(trainingSet, 4, axis=1)
trainingSet = np.delete(trainingSet, 2, axis=1)
trainingSet = np.delete(trainingSet, 1, axis=1)
trainingSet = np.delete(trainingSet, 0, axis=1)

testingSet = testingSet.astype(float)
trainingSet = trainingSet.astype(float)

for i in range (99999, 0, -1):
    for j in range(0, const_columns()):
        trainingSet[i, j] = float(trainingSet[i, j])
        if (np.isinf(trainingSet[i, j]) or np.isnan(trainingSet[i, j])):
            #print("Dropping (",i,",",j,")",testingSet[i, j])
            trainingSet = np.delete(trainingSet, i, axis=0)
            #testingSet[i, j] = 1.7*np.e**308
            
for i in range (59999,0,-1):
    for j in range(0, const_columns()):
        testingSet[i, j] = float(testingSet[i, j])
        if (np.isinf(testingSet[i, j]) or np.isnan(testingSet[i, j])):
            #print("Dropping (",i,",",j,")",testingSet[i, j])
            testingSet = np.delete(testingSet, i, axis=0)
            #testingSet[i, j] = 1.7*np.e**308
            
#print(testingSet[:,77])            
y_test = testingSet[:, const_columns()].astype(int)
y_training = trainingSet[:, const_columns()].astype(int)

print(y_test)

testingSet = np.delete(testingSet, const_columns(), axis=1)
trainingSet = np.delete(trainingSet, const_columns(), axis=1)

[1 1 1 ... 0 0 0]


In [43]:
print(testingSet.shape)
print(np.sum(y_test)/testingSet.shape[0])

(59633, 77)
0.17198531014706622


In [44]:
clf_name = 'KNN'
#clf = KNN(contamination = 0.17293680147540488)
clf = XGBOD()
clf.fit(trainingSet, y_training) 

C:\Users\caioa\Anaconda3\lib\site-packages\pyod\models\base.py:344: UserWarning: y should not be presented in unsupervised learning.
  "y should not be presented in unsupervised learning.")
C:\Users\caioa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\Users\caioa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
C:\Users\caioa\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change

XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0), KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='mean',
  metric='minkowski', metric_params=None, n_jobs=1, n_...ax_features=1.0,
    max_samples='auto', n_estimators=250, n_jobs=1, random_state=0,
    verbose=0)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
   nthread=None, objective='binary:logistic', random_state=0, reg_alpha=0,
   reg_lambda=1, scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

In [45]:
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

y_test_pred = clf.predict(testingSet)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(testingSet)  # outlier scores

y_test_pred

array([1, 1, 1, ..., 0, 0, 0])

In [46]:
from pyod.utils.data import evaluate_print

print("\nOn Test Data:")
truePositive = 0
trueNegative = 0
falsePositive = 0
falseNegative = 0

#print(y_test.shape)
#print(y_test_pred.shape)

for i in range(y_test.size):
    if(y_test[i] == 1 and y_test_pred[i] == 1):
        truePositive = truePositive+1
    elif (y_test[i] == 0 and y_test_pred[i] == 0):
        trueNegative = trueNegative+1
    elif (y_test[i] == 0 and y_test_pred[i] == 1):
        falsePositive = falsePositive+1
    else:
        falseNegative = falseNegative+1

print(truePositive, " | ", falsePositive)
print(falseNegative, " | ", trueNegative,"\n")

evaluate_print("XGBOD", y_test, y_test_scores)


On Test Data:
(59633,)
(59633,)
10247  |  64
9  |  49313 

XGBOD ROC:1.0, precision @ rank n:0.9971
